In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd
import numpy as np

#### Search your course and country in the studyportal.com, then copy the results link and paste here
It may take much longer according to your internet condition and server trafic so wait until the page is loaded completely.

In [7]:
url = input('Url: ') 
driver = webdriver.Safari()
driver.get(url)
time.sleep(3)

Url: https://www.mastersportal.com/search/master/data-science-big-data/netherlands


In [8]:
field = url.split('/')[-2]
country = url.split('/')[-1]
country

'netherlands'

In [9]:
no_change_counter = 0
old_height = 0
while True:
    # Try
    new_height = driver.execute_script("return document.body.scrollHeight") 
    driver.execute_script("window.scrollTo(0,{})".format(new_height))
    time.sleep(1)
    if new_height > old_height:
        old_height = new_height
    else:
        time.sleep(3)
        no_change_counter += 1
        
    if no_change_counter >= 3:
        break

In [11]:
results = driver.find_elements(By.XPATH, value = '//a[@class="SearchStudyCard js-bestFitStudycard"]')
len(results)

44

In [12]:
names_list = []
fees_list = []
fees_desc_list = []
duration = []
organization = []
location = []
summary = []
for i in range(len(results)):
    names_list.append(results[i].find_element(By.CLASS_NAME, value='StudyName').text)
    try:
        fees_list.append(int(results[i].find_element(By.CLASS_NAME, value='TuitionValue').text.split(' ')[0].replace(',', '')))
        fees_desc_list.append(results[i].find_element(By.CLASS_NAME, value='TuitionValue').text.split(' ', 1)[1])
        duration.append(results[i].find_element(By.CLASS_NAME, value='DurationValue').text)
        organization.append(results[i].find_element(By.CLASS_NAME, value='OrganisationName').text)
        location.append(results[i].find_element(By.CLASS_NAME, value='OrganisationLocation').text)
        summary.append(results[i].find_element(By.CLASS_NAME, value='Summary').text)
    except:
        fees_list.append(np.nan)
        fees_desc_list.append(np.nan)
        duration.append(np.nan)
        organization.append(np.nan)
        location.append(np.nan)
        summary.append(np.nan)


In [13]:
df = pd.DataFrame(list(zip(names_list, fees_list, fees_desc_list, duration, organization,location, summary)),
                 columns=['Courses', 'Fees', 'Fees Type', 'Duration', 'Organization', 'Location', 'Course Description'], index=None)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Courses             44 non-null     object
 1   Fees                44 non-null     int64 
 2   Fees Type           44 non-null     object
 3   Duration            44 non-null     object
 4   Organization        44 non-null     object
 5   Location            44 non-null     object
 6   Course Description  44 non-null     object
dtypes: int64(1), object(6)
memory usage: 2.5+ KB


In [15]:
df

,Courses,Fees,Fees Type,Duration,Organization,Location,Course Description
0,Data Science,2314,EUR / year,2 years,Radboud University,"Nijmegen, Netherlands",The Master's specialisation in Data Science at...
1,Smart Mobility Data Science and Analytics,4000,EUR / year,2 years,EIT Urban Mobility Master School,Multiple locations,Smart Mobility Data Science and Analytics at E...
2,Business Analytics and Operations Research,2253,EUR / year,1 year,Tilburg University,"Tilburg, Netherlands",Do you like working with data and mathematics?...
3,Data Science for Food and Health,2314,EUR / year,2 years,Wageningen University and Research,"Wageningen, Netherlands",You will be able to integrate data science kno...
4,Data Science (DSC),5000,EUR / year,2 years,EIT Digital,Multiple locations,The interdisciplinary Data Science (DSC) progr...
5,Digital Technology Engineering,1104,EUR / year,2 years,Fontys University of Applied Sciences,"Eindhoven, Netherlands",The Master of Science (MSc) in Digital Technol...
6,Data Science and Society,2253,EUR / year,1 year,Tilburg University,"Tilburg, Netherlands",Become a Data Analyst and provide solutions fo...
7,Data Science and Artificial Intelligence,2209,EUR / year,2 years,Eindhoven University of Technology (TU/e),"Eindhoven, Netherlands",Data Science and Artificial Intelligence from ...
8,Data Science in Engineering,2209,EUR / year,2 years,Eindhoven University of Technology (TU/e),"Eindhoven, Netherlands",The Data Science in Engineering programme from...
9,Business Intelligence and Smart Services,2209,EUR / year,1 year,Maastricht University,"Maastricht, Netherlands",Do you look at a mountain of data and see an o...


In [16]:
df.to_excel('{}_{}.xlsx'.format(field, country))